# Creates a database of mushrooms from scraping www.wildfooduk.com

In [59]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os

In [60]:
shroom_DB = "https://www.wildfooduk.com/mushroom-guide/?mushroom_orderby=latin_name&mushroom_order=DESC"

result = requests.get(shroom_DB)

soup = BeautifulSoup(result.content, "html.parser")

table = soup.find('table', class_='mushroom-table')

shrooms_scientific = []; shrooms_common = []
shrooms_start = []; shrooms_end = []
shrooms_edible = []; shrooms_seasons = []

for row in table.tbody.find_all('tr'):
    
    shroom_image, shroom_common, shroom_scientific, shroom_info, shroom_season_start, shroom_season_end = row.find_all('td')
        
    info = [info['alt'] for info in shroom_info.find_all('img')]    
    
    shrooms_edible.append(info[0])
    shrooms_seasons.append(info[1:])
    
    shrooms_start.append(shroom_season_start.text.replace('\nSeason Start : ', ''))
    shrooms_end.append(shroom_season_end.text.replace('\nSeason End : ', ''))
    
    shrooms_scientific.append(shroom_scientific.get_text().strip())
    shrooms_common.append(shroom_common.get_text().strip())

#### Renaming Boletus and Hygrocybe varieties classes

In [61]:
shrooms_scientific = [shroom.replace('Boletus / ', '') for shroom in shrooms_scientific]
shrooms_scientific = [shroom.replace('Hygrocybe / ', '') for shroom in shrooms_scientific]

#### Removing other varieties of species from search, may cause issues elsewhere

#### Create dataframe

In [62]:
d = {'scientific': shrooms_scientific, 
     'common': shrooms_common, 
     'season start': shrooms_start, 
     'season end': shrooms_end, 
     'Type': shrooms_edible,
     'shrooms seasons': shrooms_seasons}

df = pd.DataFrame(data=d)
df.head(5)

,scientific,common,season start,season end,Type,shrooms seasons
0,Tuber aestivum,Truffles,Aug,Nov,Edible,"[Autumn, Summer]"
1,Tricholomopsis rutilans,Plums And Custard,Jul,Nov,Inedible,"[Autumn, Summer]"
2,Tricholoma terreum,The Grey Knight,Aug,Oct,Poisonous,"[Autumn, Summer]"
3,Stropharia caerulea,Blue Roundhead,Aug,Dec,Poisonous,"[Autumn, Winter]"
4,Sparassis crispa,Cauliflower Fungus,Aug,Nov,Edible,"[Autumn, Summer]"


In [63]:
df = df.replace('Scleroderma citrinum, areolatum, verrucosum', 
                          'Scleroderma citrinum')

df = df.replace('Inocybe geophylla', 
                          'Inocybe geophylla"  -"lilacina')

df = df.replace('Cantherellus tubaeformis', 
                          'Cantharellus tubaeformis')

df = df.replace('Boletus / Leccinum versipellis', 
                          'Leccinum versipelle')

df = df.replace('Amanita citrina var citrina', 'Amanita citrina')

df = df[df.scientific != 'Amanita citrina var alba']

In [64]:
df.to_csv(os.getcwd() + '\\Shroom_DB.csv')

In [65]:
df.head()

,scientific,common,season start,season end,Type,shrooms seasons
0,Tuber aestivum,Truffles,Aug,Nov,Edible,"[Autumn, Summer]"
1,Tricholomopsis rutilans,Plums And Custard,Jul,Nov,Inedible,"[Autumn, Summer]"
2,Tricholoma terreum,The Grey Knight,Aug,Oct,Poisonous,"[Autumn, Summer]"
3,Stropharia caerulea,Blue Roundhead,Aug,Dec,Poisonous,"[Autumn, Winter]"
4,Sparassis crispa,Cauliflower Fungus,Aug,Nov,Edible,"[Autumn, Summer]"


In [78]:
df[df.scientific == 'clitopilus_prunulus'.capitalize().replace('_', ' ')]

,scientific,common,season start,season end,Type,shrooms seasons
96,Clitopilus prunulus,The Miller,Jun,Nov,Edible,"[Autumn, Summer]"


In [83]:
prediction = 140

df.iloc[prediction]['season start']

'Aug '

In [117]:
scientific = df.iloc[prediction]['scientific']
common = str(df.iloc[prediction]['common'])
start = str(df.iloc[prediction]['season start'])
end = str(df.iloc[prediction]['season end'])
edible = str(df.iloc[prediction]['Type'])

caption = f'{scientific}\nCommon name: {common}\n{edible}\nSeason: {start} - {end}'

print(caption)

Aleuria aurantia
Common name: Orange Peel Fungus
Edible
Season: Aug  - Nov 


In [118]:
scientific

'Aleuria aurantia'